<a href="https://colab.research.google.com/github/MaadElGali/mjDrmUGvs9wf2Jff/blob/main/TalentSourcingJobs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3 Potential Talents

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument  #word & document embeddings
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import xgboost as xgb


In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 68.1 MB/s eta 0:00:00


In [ ]:
#Loading a dataset into Google Colab from the local machine
from google.colab import files
uploaded = files.upload()

Saving potential-talents - Aspiring human resources - seeking human resources.csv to potential-talents - Aspiring human resources - seeking human resources.csv


In [ ]:
#Load the dataset

df =pd.read_csv('potential-talents - Aspiring human resources - seeking human resources.csv')

In [ ]:
df

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN


In [ ]:
#Check for missing data
df.isnull().sum()

,0
id,0
job_title,0
location,0
connection,0
fit,104


In [ ]:
#Find all unique values in the 'connection' column
unique_values = df['connection'].unique()
unique_values

array(['85', '500+ ', '44', '1', '61', '2', '390', '57', '82', '5', '7',
       '16', '212', '409', '52', '455', '174', '268', '50', '4', '40',
       '18', '349', '155', '39', '64', '9', '415', '19', '71', '48',
       '103', '49'], dtype=object)

In [ ]:
#Count the number of occurrences of '500+ ' in the 'connection' column
count_500_plus = df['connection'].value_counts().get('500+ ', 0)
count_500_plus

np.int64(44)

In [ ]:
#Display all rows where the 'connection' column has '500+'
rows_with_500_plus = df[df['connection'] == '500+ ']
rows_with_500_plus

,id,job_title,location,connection,fit
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
7,8,HR Senior Specialist,San Francisco Bay Area,500+,NaN
9,10,Seeking Human Resources HRIS and Generalist Po...,Greater Philadelphia Area,500+,NaN
11,12,"SVP, CHRO, Marketing & Communications, CSR Off...","Houston, Texas Area",500+,NaN
12,13,Human Resources Coordinator at InterContinenta...,"Atlanta, Georgia",500+,NaN
15,16,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
17,18,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
19,20,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN


In [ ]:
#Step 1
#Keep only the columns 'job_title' and 'connection'
df_small = df[['job_title', 'connection']].copy()

In [ ]:
df_small

,job_title,connection
0,2019 C.T. Bauer College of Business Graduate (...,85
1,Native English Teacher at EPIK (English Progra...,500+
2,Aspiring Human Resources Professional,44
3,People Development Coordinator at Ryan,500+
4,Advisory Board Member at Celal Bayar University,500+
...,...,...
99,Aspiring Human Resources Manager | Graduating ...,103
100,Human Resources Generalist at Loparex,500+
101,Business Intelligence and Analytics at Travelers,49
102,Always set them up for Success,500+


In [ ]:
#Convert all instances of '500+ ' to the numeric value '500' and ensure numeric
df_small['connection'] = df_small['connection'].apply(lambda x: 500 if x == '500+ ' else int(x))


In [ ]:
df_small

,job_title,connection
0,2019 C.T. Bauer College of Business Graduate (...,85
1,Native English Teacher at EPIK (English Progra...,500
2,Aspiring Human Resources Professional,44
3,People Development Coordinator at Ryan,500
4,Advisory Board Member at Celal Bayar University,500
...,...,...
99,Aspiring Human Resources Manager | Graduating ...,103
100,Human Resources Generalist at Loparex,500
101,Business Intelligence and Analytics at Travelers,49
102,Always set them up for Success,500


In [ ]:
#Calculate occurrences of each unique value in the 'connection' column
occurrences = df_small['connection'].value_counts()
occurrences

,count
connection,
500,44
85,7
61,7
44,6
1,5
2,4
390,2
57,2
7,2


In [ ]:
#Step2
#Doc2Vec embeddings for "Aspiring human resources" (w1) and each job_title
#Prepare doc2vec training corpus from job_title strings
documents = [TaggedDocument(words=jt.lower().split(), tags=[str(i)]) for i, jt in enumerate(df_small['job_title'].astype(str))]
d2v_model = Doc2Vec(vector_size=100, min_count=1, epochs=40, workers=4, seed=42)
d2v_model.build_vocab(documents)
d2v_model.train(documents, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

In [ ]:
#Get the embeddings for each job_title
def doc2vec_embed(text):
    return d2v_model.infer_vector(str(text).lower().split())

job_embeds_d2v = np.vstack([doc2vec_embed(t) for t in df_small['job_title']])

In [ ]:
#Embedding for query "Aspiring human resources"
w1_d2v = doc2vec_embed("Aspiring human resources")

In [ ]:
w1_d2v

array([-5.58023667e-03,  1.03386668e-02,  2.55747861e-03,  6.79179514e-03,
       -5.80644747e-03,  1.00249695e-02,  6.45891763e-04,  3.08958371e-03,
       -6.76247803e-03, -4.65260725e-03, -8.03881232e-03, -7.09798536e-04,
        3.05372174e-03,  4.86188661e-03, -7.54986424e-03, -1.04932440e-03,
       -8.52812431e-04, -1.44956382e-02, -3.10545583e-04, -1.06463500e-03,
        6.79852266e-04,  8.09429400e-03,  5.70608163e-03,  8.15543346e-03,
        9.34052560e-03,  3.58668761e-03, -6.64673746e-03,  8.72547273e-03,
       -9.50369518e-03, -7.39952875e-03, -1.20330481e-02, -1.77106482e-03,
       -3.02481698e-03, -3.77125863e-04, -6.52749077e-05, -1.88970205e-03,
        5.80549054e-03, -4.38713701e-03,  3.73610388e-03,  5.30777220e-03,
        6.09999895e-03, -5.31103555e-03,  3.87318712e-03, -2.62398575e-03,
        1.41268526e-03,  5.10644459e-04,  4.67125280e-03,  1.17838141e-02,
        5.13205631e-03, -4.96674934e-03,  6.25423971e-04, -9.58624668e-03,
       -6.49181707e-03, -

In [ ]:
#Step 3
#Cosine similarity between w1 and each job_title (doc2vec)
cos_sim_d2v = cosine_similarity(job_embeds_d2v, w1_d2v.reshape(1, -1)).reshape(-1)

In [ ]:
cos_sim_d2v

array([0.8636282 , 0.86133325, 0.8445324 , 0.86465013, 0.8635489 ,
       0.8462012 , 0.8634516 , 0.86534154, 0.86380947, 0.85929596,
       0.861439  , 0.8632213 , 0.8678668 , 0.8637538 , 0.8637208 ,
       0.8613476 , 0.8533896 , 0.8643929 , 0.8637712 , 0.86169994,
       0.83225596, 0.8642633 , 0.8636946 , 0.83365005, 0.8642175 ,
       0.8652589 , 0.8645141 , 0.8575751 , 0.86451757, 0.857026  ,
       0.8638319 , 0.86170846, 0.8482381 , 0.86449933, 0.86358714,
       0.8482802 , 0.8637494 , 0.8655288 , 0.86411476, 0.8594748 ,
       0.8621626 , 0.8630827 , 0.86753607, 0.86394405, 0.86127865,
       0.81847286, 0.86444557, 0.863371  , 0.8483741 , 0.8636518 ,
       0.86576813, 0.8633604 , 0.85936224, 0.8626147 , 0.8631641 ,
       0.8679841 , 0.86369073, 0.8405515 , 0.8649771 , 0.808501  ,
       0.86528057, 0.8592943 , 0.8621936 , 0.8630086 , 0.8679781 ,
       0.86366236, 0.86322117, 0.8652377 , 0.8619087 , 0.8622066 ,
       0.86170375, 0.8623783 , 0.8634449 , 0.74662435, 0.86286

In [ ]:
#Step 4
#Build dataframe with job_title, connection, cosine_similarity
df_d2v = df_small[['job_title', 'connection']].copy()
df_d2v['cosine_similarity'] = cos_sim_d2v
df_d2v['connection'] = df_small['connection']

In [ ]:
df_d2v

,job_title,connection,cosine_similarity
0,2019 C.T. Bauer College of Business Graduate (...,85,0.863628
1,Native English Teacher at EPIK (English Progra...,500,0.861333
2,Aspiring Human Resources Professional,44,0.844532
3,People Development Coordinator at Ryan,500,0.864650
4,Advisory Board Member at Celal Bayar University,500,0.863549
...,...,...,...
99,Aspiring Human Resources Manager | Graduating ...,103,0.862939
100,Human Resources Generalist at Loparex,500,0.860861
101,Business Intelligence and Analytics at Travelers,49,0.864128
102,Always set them up for Success,500,0.862526


In [ ]:
#Step 5
#Scale the connection column to 0-1 (using MinMaxScaler)
scaler = MinMaxScaler()
df_d2v['scaled_connection'] = scaler.fit_transform(df_d2v[['connection']])

In [ ]:
df_d2v

,job_title,connection,cosine_similarity,scaled_connection
0,2019 C.T. Bauer College of Business Graduate (...,85,0.863628,0.168337
1,Native English Teacher at EPIK (English Progra...,500,0.861333,1.000000
2,Aspiring Human Resources Professional,44,0.844532,0.086172
3,People Development Coordinator at Ryan,500,0.864650,1.000000
4,Advisory Board Member at Celal Bayar University,500,0.863549,1.000000
...,...,...,...,...
99,Aspiring Human Resources Manager | Graduating ...,103,0.862939,0.204409
100,Human Resources Generalist at Loparex,500,0.860861,1.000000
101,Business Intelligence and Analytics at Travelers,49,0.864128,0.096192
102,Always set them up for Success,500,0.862526,1.000000


In [ ]:
#Step 6
#Create the ranking based on the following = 0.8*cosine_similarity + 0.2*scaled_connection
df_d2v['ranking'] = 0.8 * df_d2v['cosine_similarity'] + 0.2 * df_d2v['scaled_connection']

In [ ]:
#Step 7
#Sort the df descending by ranking
df_d2v_sorted = df_d2v.sort_values('ranking', ascending=False).reset_index(drop=True)

In [ ]:
df_d2v_sorted

,job_title,connection,cosine_similarity,scaled_connection,ranking
0,Human Resources Coordinator at InterContinenta...,500,0.867984,1.000000,0.894387
1,Human Resources Coordinator at InterContinenta...,500,0.867978,1.000000,0.894382
2,Human Resources Coordinator at InterContinenta...,500,0.867867,1.000000,0.894293
3,Human Resources Coordinator at InterContinenta...,500,0.867536,1.000000,0.894029
4,HR Senior Specialist,500,0.865768,1.000000,0.892614
...,...,...,...,...,...
99,Aspiring Human Resources Specialist,1,0.846201,0.000000,0.676961
100,Aspiring Human Resources Professional,44,0.818473,0.086172,0.672013
101,Aspiring Human Resources Specialist,1,0.833650,0.000000,0.666920
102,Aspiring Human Resources Specialist,1,0.808501,0.000000,0.646801


In [ ]:
#df_d2v_sorted now holds the results for Doc2Vec-only approach
#Get the top-n candidates:
def top_n_d2v(n=10):
    return df_d2v_sorted.head(n)

In [ ]:
#Load the sentence-BERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Create the sentence-BERT embeddings for query and job_title
job_titles = df_small['job_title'].astype(str).tolist()
job_embeds_sbert = sbert_model.encode(job_titles, convert_to_numpy=True, normalize_embeddings=False)
w1_sbert = sbert_model.encode(["Aspiring human resources"], convert_to_numpy=True)[0]

In [ ]:
#Calculate the cosine similarity between SBERT query and each job_title
cos_sim_sbert = cosine_similarity(job_embeds_sbert, w1_sbert.reshape(1, -1)).reshape(-1)

In [ ]:
#Combine Doc2Vec and SBERT
#Combine the similarity scores from both embeddings
#Normalize both similarity arrays to 0-1 before combining
def minmax_arr(a):
    a = np.array(a, dtype=float)
    if a.max() == a.min():
        return np.zeros_like(a)
    return (a - a.min()) / (a.max() - a.min())

sim_d2v_norm = minmax_arr(cos_sim_d2v)
sim_sbert_norm = minmax_arr(cos_sim_sbert)

In [ ]:
#Weighted combination of similarities
#Here I give 0.6 to SBERT and 0.4 to Doc2Vec
combined_sim = 0.6 * sim_sbert_norm + 0.4 * sim_d2v_norm

df_combined = df_small[['job_title', 'connection']].copy()
df_combined['cosine_similarity_doc2vec'] = cos_sim_d2v
df_combined['cosine_similarity_sbert'] = cos_sim_sbert
df_combined['combined_similarity'] = combined_sim
df_combined['connection'] = df_small['connection']
df_combined['scaled_connection'] = scaler.fit_transform(df_combined[['connection']])

In [ ]:
#Calculate the final ranking using the combined similarity and scaled_connection (0.8 weight to similarity)
df_combined['ranking'] = 0.8 * df_combined['combined_similarity'] + 0.2 * df_combined['scaled_connection']
df_combined_sorted = df_combined.sort_values('ranking', ascending=False).reset_index(drop=True)

In [ ]:
#Retrieve Top-n for combined:
def top_n_combined(n=10):
    return df_combined_sorted.head(n)

# End results:
# - df_d2v_sorted: Doc2Vec-only ranking
# - df_sbert_sorted: Sentence-BERT-only ranking
# - df_combined_sorted: Combined Doc2Vec+SBERT ranking

In [ ]:
#Print the top 5 from combined

df_combined_sorted.head(5)

,job_title,connection,cosine_similarity_doc2vec,cosine_similarity_sbert,combined_similarity,scaled_connection,ranking
0,"Human Resources, Staffing and Recruiting Profe...",500,0.863221,0.745484,0.842303,1.0,0.873843
1,Seeking Human Resources HRIS and Generalist Po...,500,0.859475,0.744711,0.829418,1.0,0.863534
2,Seeking Human Resources HRIS and Generalist Po...,500,0.859362,0.744711,0.829047,1.0,0.863237
3,Aspiring Human Resources Management student se...,500,0.864518,0.720099,0.828934,1.0,0.863147
4,Aspiring Human Resources Management student se...,500,0.864514,0.720099,0.828923,1.0,0.863138


In [ ]:
#Add the 'star' column: Top 20 candidates get a value of 1, others get 0
df_combined_sorted['star'] = 0
df_combined_sorted.loc[:19, 'star'] = 1  # Top 20 (0-based index implies 0-19)

#Now the dataframe df_combined_sorted includes a 'star' column

In [ ]:
df_combined_sorted

,job_title,connection,cosine_similarity_doc2vec,cosine_similarity_sbert,combined_similarity,scaled_connection,ranking,star
0,"Human Resources, Staffing and Recruiting Profe...",500,0.863221,0.745484,0.842303,1.000000,0.873843,1
1,Seeking Human Resources HRIS and Generalist Po...,500,0.859475,0.744711,0.829418,1.000000,0.863534,1
2,Seeking Human Resources HRIS and Generalist Po...,500,0.859362,0.744711,0.829047,1.000000,0.863237,1
3,Aspiring Human Resources Management student se...,500,0.864518,0.720099,0.828934,1.000000,0.863147,1
4,Aspiring Human Resources Management student se...,500,0.864514,0.720099,0.828923,1.000000,0.863138,1
...,...,...,...,...,...,...,...,...
99,Human Resources Professional,16,0.746624,0.794648,0.492169,0.030060,0.399747,0
100,Bachelor of Science in Biology from Victoria U...,40,0.862289,0.222139,0.475523,0.078156,0.396050,0
101,Admissions Representative at Community medical...,9,0.863330,0.161050,0.436501,0.016032,0.352407,0
102,Student,4,0.847655,0.238388,0.438581,0.006012,0.352068,0


In [ ]:
#Define features and target
X = df_combined_sorted[['combined_similarity', 'scaled_connection']].values
y = df_combined_sorted['ranking'].values

In [ ]:
#Split the data into train & test sets split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Initiate the LTR model
ltr_model = xgb.XGBRanker(
    objective='rank:pairwise',  # Use appropriate LTR objective
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    random_state=42
)

In [ ]:
#Fit the LTR model
ltr_model.fit(X_train, y_train, group=[len(X_train)])

XGBRanker(base_score=None, booster=None, callbacks=None, colsample_bylevel=None,
          colsample_bynode=None, colsample_bytree=None, device=None,
          early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, feature_weights=None,
          gamma=None, grow_policy=None, importance_type=None,
          interaction_constraints=None, learning_rate=0.1, max_bin=None,
          max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=None,
          max_depth=6, max_leaves=None, min_child_weight=None, missing=nan,
          monotone_constraints=None, multi_strategy=None, n_estimators=100,
          n_jobs=None, num_parallel_tree=None, ...)

In [ ]:
#Evaluate the LTR model (ranking score predictions)
predictions = ltr_model.predict(X_test)
#Predicted ranking scores
print(predictions)

[ 0.77713686 -1.6377474  -1.6377474  -1.6377474  -1.9991139  -2.5911617
 -2.57452    -1.6471125   1.2762861   1.9203      0.68986243 -0.24614617
 -2.1452544  -2.5911617  -1.6377474   1.802351   -1.6377474   0.3375098
  1.1643095  -0.53874207 -2.5911617 ]
